# AS05
這裡有兩題看似要你寫Crawler，但事實上要怎麼Crawl或要怎麼Parse HTML老師都幫你寫好了。程式碼都是完整的，但卻只是某個模組功能的。所以你在這個作業要做的，實際上是看懂這些程式碼，並把這些程式碼兜起來。

這個作業要考驗你的是耐心看懂別人的程式碼，和重組別人的程式碼。

# AS05_1. Getting ibon store address
* 這題的目標是取得所有ibon生活便利站的所在地址。http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0。
* 我已經幫你寫好，怎麼從上述網頁取得所有縣市名稱（`get()` + `BeautifulSoup`如**Get all counties***）。我也幫你寫好，怎麼取得某一個縣市的所有生活便利站（`post()` + `BeautifulSoup` 如**Get stores of a county**）。



In [1]:
import requests

from bs4 import BeautifulSoup

## Get all counties

In [2]:
url_counties = 'http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0'
res = requests.get(url_counties)
soup = BeautifulSoup(res.text, "html.parser")
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n <head>\n  <!-- viewport -->\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta charset="utf-8"/>\n  <title id="titleNaming">\n   ibon 便利生活站 - 門市查詢-網頁版\n  </title>\n  <link href="css/pcstyle.css" rel="stylesheet" type="text/css"/>\n  <link href="css/font-awesome.min.css" rel="stylesheet"/>\n  <!-- jQuery -->\n  <script src="js/jquery-3.5.1.min.js">\n  </script>\n  <script src="js/jquery-migrate-1.4.1.min.js">\n  </script>\n  <script src="js/jquery-migrate-3.3.2.min.js">\n  </script>\n  <script src="js/selectivizr-min.js">\n  </script>\n  <script src="js/script.js">\n  </script>\n  <script src="js/commonutil.js">\n  </script>\n  <!-- 解決IE7~8看不懂html5的標籤 -->\n  <!--[if lt IE 9]>\r\n<script src=\x0b"http://html5shim.googlecode.com/svn/trunk/html5.js">\r\n</script>\r\n<![endif]-->\n  <style>\n   .subContainer {\r\n            display: none;\r\n 


- Selecting data by CSS selector https://stackoverflow.com/questions/24801548/how-to-use-css-selectors-to-retrieve-specific-links-lying-in-some-class-using-be
- Selecting node with id: https://stackoverflow.com/questions/56860183/how-to-find-allid-from-a-div-with-beautiful-soup-in-python/56860413

In [3]:
counties = []
for op in soup.select("#Class1 option"):
    print(op.text)
    counties.append(op.text)

台北市
新北市
基隆市
宜蘭縣
桃園市
新竹市
新竹縣
苗栗縣
台中市
彰化縣
南投縣
雲林縣
嘉義市
嘉義縣
台南市
高雄市
屏東縣
花蓮縣
台東縣
澎湖縣
金門縣
連江縣
南海諸島


https://stackoverflow.com/questions/50633050/scrape-tables-into-dataframe-with-beautifulsoup

## Get stores of a county

In [4]:
payloads = {'strTargetField':'COUNTY', 
            'strKeyWords':'台北市'}
url_ibon = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
raw = requests.post(url_ibon, data=payloads)
soup = BeautifulSoup(raw.text, "html.parser")

In [5]:
import pandas as pd
table_rows = soup.find('table').find_all('tr')
cname = ['sid', 'sname', 'addr']
rows = []
for tr in table_rows[1:]:
    tds = tr.find_all('td')
    row = []
    for td in tds:
        row.append(td.text.strip())
    rows.append(row)
pd.DataFrame(rows, columns=cname)

,sid,sname,addr
0,110817,千翔,台北市中正區許昌街17號1樓
1,111278,前港,台北市士林區後港街98之1號2號1樓
2,111636,江東,台北市中山區長安東路二段43號45-1號1樓
3,112684,塔優,台北市松山區撫遠街197號199號
4,113089,秀湖,台北市內湖區成功路四段359號1樓
...,...,...,...
861,996523,篷萊,台北市大同區寧夏路131-1號
862,996637,和業,台北市北投區中和街10號
863,996741,元大,台北市中山區遼寧街110號1樓
864,997571,聚鑫,台北市士林區延平北路六段350號


## (YOUR CODE HERE) Combine them
Output should be 
```
COUNTY	Num of Stores
台北市 	 855
新北市 	 1043
基隆市 	 80
宜蘭縣 	 108
桃園市 	 697
新竹市 	 154
新竹縣 	 183
苗栗縣 	 135
台中市 	 755
彰化縣 	 225
南投縣 	 100
雲林縣 	 120
嘉義市 	 67
嘉義縣 	 103
台南市 	 512
高雄市 	 609
屏東縣 	 163
花蓮縣 	 99
台東縣 	 60
澎湖縣 	 31
金門縣 	 25
連江縣 	 12
南海諸島 	
```

In [12]:
rows = []
url_ibon = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
# YOUR CODE GOES FROM HERE

#先建好每個城市的payload
payload = []
for n in range(len(counties)):
    payload.append({'strTargetField':'COUNTY', 'strKeyWords':counties[n]})
# print(payload)

# 開一個list of dict用來output上面那些
table_county = []

#開始爬資料、並存進rows
for x in payload:
    raw = requests.post(url_ibon, data=x)
    soup = BeautifulSoup(raw.text, "html.parser")
    table_rows = soup.find('table').find_all('tr')    
    
    for tr in table_rows[1:]:
        tds = tr.find_all('td')
        row = []
        for td in tds:
            row.append(td.text.strip())
        rows.append(row)
    num = len(table_rows)-1
    table_county.append({'COUNTY':x['strKeyWords'], 'Num of Stores':num})

pd.DataFrame(table_county)

,COUNTY,Num of Stores
0,台北市,866
1,新北市,1073
2,基隆市,83
3,宜蘭縣,112
4,桃園市,722
5,新竹市,156
6,新竹縣,191
7,苗栗縣,141
8,台中市,785
9,彰化縣,234


## AS04_1 Output Verification

### Verification 1_1
```
sid	sname	addr
0	110817	千翔	台北市中正區許昌街17號1樓
1	111278	前港	台北市士林區後港街98之1號2號1樓
2	111636	江東	台北市中山區長安東路二段43號45-1號1樓
3	112684	塔優	台北市松山區撫遠街197號199號
4	113089	秀湖	台北市內湖區成功路四段359號1樓
...	...	...	...
6132	950376	南竿	連江縣南竿鄉清水村100號
6133	952512	馬港	連江縣南竿鄉馬祖村中山路58號60號
6134	952523	北竿	連江縣北竿鄉塘岐村中山路193號195號
6135	991322	中隴	連江縣南竿鄉介壽村3鄰48-2號
6136	無符合條件的門市資料，請重新定義條件	None	None
6137 rows × 3 columns
```

In [13]:
cname = ['sid', 'sname', 'addr']
ibon_df = pd.DataFrame(rows, columns=cname)
ibon_df

,sid,sname,addr
0,110817,千翔,台北市中正區許昌街17號1樓
1,111278,前港,台北市士林區後港街98之1號2號1樓
2,111636,江東,台北市中山區長安東路二段43號45-1號1樓
3,112684,塔優,台北市松山區撫遠街197號199號
4,113089,秀湖,台北市內湖區成功路四段359號1樓
...,...,...,...
6321,950376,南竿,連江縣南竿鄉清水村100號
6322,952512,馬港,連江縣南竿鄉馬祖村中山路58號60號
6323,952523,北竿,連江縣北竿鄉塘岐村中山路193號195號
6324,991322,中隴,連江縣南竿鄉介壽村3鄰48-2號


### Verification 1_2

In [14]:
ibon_df.shape

(6326, 3)

# AS05_2 Scraping PTT comments
- 撈取Boy-Girl版最新的10頁文章（並非10則文章，是10頁的文章，約150則左右）。
- 撈取的同時，請稍微修改下面的Code以抓回Comments

## Getting 10 pages of post list
- `lastpage` 不見得是5303，撈得到就好。重點是撈10個post list頁面

In [47]:
import requests
from bs4 import BeautifulSoup

In [49]:
pre = 'https://www.ptt.cc'
links = []
lastpage = 5308
for i in range(lastpage, lastpage-10, -1):
    url = 'https://www.ptt.cc/bbs/Boy-Girl/index{}.html'.format(i)
    response = requests.get(url, timeout=(1, 2))
    soup = BeautifulSoup(response.text, 'html.parser')
    for div in soup.find_all(class_='r-ent'):
        try:
            links.append(pre + div.find(class_='title').a['href'])
        except:
            pass
    print(i, "\t", len(links))

5308 	 20
5307 	 40
5306 	 60
5305 	 80
5304 	 100
5303 	 120
5302 	 140
5301 	 160
5300 	 180
5299 	 200


## Getting comments of a specific post
- 撈到的links數也不見得會是165

In [50]:
print(links[0])
print(len(links))

https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A.CF5.html
200


In [51]:
res = requests.get(links[0], timeout=(1, 2))
soup = BeautifulSoup(res.text, "html.parser")

comments = []
for push in soup.find_all(class_="push"):
    push_tag = push.find(class_='push-tag').text
    push_userid = push.find(class_='push-userid').text
    push_content = push.find(class_='push-content').text
    push_ipdatetime = push.find(class_='push-ipdatetime').text

    #print push_userid, push_content, push_ipdatetime
    comments.append({'tag': push_tag,
                     'userid':push_userid, 
                     'content':push_content, 
                     'timestamp':push_ipdatetime})
pd.DataFrame(comments)

,tag,userid,content,timestamp
0,→,Levesque,: 沒有耶，我有兩位高中同學後來又一起讀同所大學，都沒有,05/08 16:08\n
1,→,Levesque,: 自卑的狀況，都很多人追，外貌長不好看的才會自卑啦,05/08 16:08\n
2,推,rock801121,: 合校好，上課可以跟妹子傳紙條,05/08 18:28\n
3,→,rock801121,: 男校上課只能傳FHM跟寶島少年,05/08 18:30\n
4,→,yoche2000,: 當然不是所有啦 只能說有一部份的人有這情形,05/08 22:19\n


## (YOUR CODE HERE) Combine above two chunks

上面第一部分是撈10頁的posts，取得超過100個post links。
第二部分是把某一篇posts的comment用BeautifulSoup給爬梳出來，
現在你要做的事情是，把上面這兩部分結合起來，把這超過100篇的posts中的comments給抓出來。



In [54]:
from bs4 import NavigableString
all_post = []
n = 1
for link in links:
    # YOUR CODE GOES FROM HERE
    print(n)
    print(link)
    n = n+1
    res = requests.get(link, timeout=(1, 2))
    soup = BeautifulSoup(res.text, "html.parser")
    
    #內文
    title = soup.title.text
    metas = soup.find_all(class_='article-meta-value')
    author = metas[0].text
    timestamp = metas[-1].text
    content = ''
    for text in soup.find(id = 'main-content'):
        if isinstance(text, NavigableString):
            content += text.strip()
            
    #留言
    comments = []
    for push in soup.find_all(class_="push"):
        push_tag = push.find(class_='push-tag').text
        push_userid = push.find(class_='push-userid').text
        push_content = push.find(class_='push-content').text
        push_ipdatetime = push.find(class_='push-ipdatetime').text

        #print push_userid, push_content, push_ipdatetime
        comments.append({'tag': push_tag,
                         'userid':push_userid, 
                         'content':push_content, 
                         'timestamp':push_ipdatetime})
        
    all_post.append({'author':author,
                     'link':link,
                     'title':title,
                     'timestamp':timestamp,
                     'content':content,
                     'comments':comments})
    print('-'*80)    
    

1
https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A.CF5.html
--------------------------------------------------------------------------------
2
https://www.ptt.cc/bbs/Boy-Girl/M.1620461822.A.DD0.html
--------------------------------------------------------------------------------
3
https://www.ptt.cc/bbs/Boy-Girl/M.1620462243.A.212.html
--------------------------------------------------------------------------------
4
https://www.ptt.cc/bbs/Boy-Girl/M.1620463326.A.D09.html
--------------------------------------------------------------------------------
5
https://www.ptt.cc/bbs/Boy-Girl/M.1620465934.A.40E.html
--------------------------------------------------------------------------------
6
https://www.ptt.cc/bbs/Boy-Girl/M.1620467918.A.001.html
--------------------------------------------------------------------------------
7
https://www.ptt.cc/bbs/Boy-Girl/M.1620476858.A.F73.html
--------------------------------------------------------------------------------
8
https://www.ptt.cc/bbs/Bo

## MUST Run This -> Output Verification
每則貼文的結構應該是這個樣子。
```
{
    'author': '...',
    'link': '...',
    'title': '...',
    'timestamp': '...',
    'content': '...',
    'comments': [
        {
           'tag': '推 ',
           'userid': '...',
           'content': '...',
           'timestamp': '...'
        },
        {
           'tag': '推 ',
           'userid': '...',
           'content': '...',
           'timestamp': '...'
        },
        {...},
        {...},
        {...}]
}
```
範例資料如
```
{'author': 'pauljet (拜登哥不要)',
 'link': 'https://www.ptt.cc/bbs/Boy-Girl/M.1617778263.A.0D4.html',
 'title': 'Re: [求助] 老婆家裡給房子 每次吵架就',
 'timestamp': 'Wed Apr  7 14:51:01 2021',
 'content': '你也唸一唸不會離婚啦\n\n我有這種老婆要我離婚 又不是傻了\n\n先規劃生小孩差不多\n\n房子老婆出 要小心不要被趕出去都沒你的份\n\n100萬 還是自己也在用\n\n被你講得偉大得跟什麼一樣\n\n老婆不想工作也可以唸 不是都做網拍有收入嗎？\n\n你出資金她做網拍 收入還跟你對分耶 你也有拿到盈餘\n\n你還嫌棄她不做家務 真的是 家務本來就不應該出房子的人做\n\n不知道怎麼講了 都沒要求生小孩要母姓\n\n你也腦補成入贅\n\n你這篇性別互換還不被噓爆\n\n\n--',
 'comments': [{'tag': '推 ',
   'userid': 'VVizZ',
   'content': ': 鞭屍開始XDD',
   'timestamp': ' 04/07 14:54\n'},
  {'tag': '推 ',
   'userid': 'a0913',
   'content': ': 她說又不花我錢 為什麼要催她去上班<<無業還不用花先生的錢',
   'timestamp': ' 04/07 14:54\n'},
  {'tag': '→ ',
   'userid': 'a0913',
   'content': ': 根本賺到 身在福中不知福',
   'timestamp': ' 04/07 14:55\n'},
  {'tag': '→ ',
   'userid': 'bbbing',
   'content': ': 基於安全感來個共同持有如何',
   'timestamp': ' 04/07 14:57\n'},
  {'tag': '→ ',
   'userid': 'VVizZ',
   'content': ': 當然免談 2021了還在共同持有想騙誰',
   'timestamp': ' 04/07 14:59\n'}]}
  ```

### Verification 2_1

In [55]:
all_post[0]

{'author': 'yoche2000 (九龍塘麵包粉)',
 'link': 'https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A.CF5.html',
 'title': 'Re: [討論] 自己經驗來說 贊同自己小孩唸男校女校嗎 - 看板 Boy-Girl - 批踢踢實業坊',
 'timestamp': 'Sat May  8 15:29:02 2021',
 'content': '對選男校女校的人來說 重點是校風還有資源\n以一個男校仔的觀點 男女校對感情根本沒差\n\n假設今天小孩在台北 又有第一志願的實力 那一定是建北吧\n很少聽說在台北/新竹/台中/台南/高雄 這些地方成績好的學生\n為了不要讀男/女校選擇教後的志願的\n所以選這些老牌地區型第一志願的 應該只是照志願序填\n那如果在桃園 有能力當然就選武陵\n\n單性別學校比較有名的通常都是前幾志願\n連台南原男女第二志願的南二、家齊，都已經變男女合校了\n但我聽過所謂直升機父母 將小孩送到台南"唯一全女校"的私校\n這間學校出了名的保守 他的家長就是不希望女兒談戀愛所以送過去\n而這位女兒畢業讀大學 媽媽還跟過去陪讀\n所以這應該純粹是直升機的問題 跟甚麼男女感情無關\n\n高中讀台南某男校 大家都會嘴砲說因為讀男校沒女友\n但到後來才發現 那些高中沒女友的根本是自己的關係\n男校女校對高中是否談戀愛我認為影響有限\n你可以說這些高志願男女校比較乖 談戀愛的比例會比較少\n那這或許還說過去 但這不是因為男女校的關係阿\n純粹是 成績好的族群中比較多乖的 第一志願這種人當然比較多啦\n\n我母校有四大當社 這些玩社團比較兇的往往都會組聯盟\n地緣性較高的幾個學校組起來 常常一起聯合社課之類的\n要認識異性的機會還不夠多嗎 外宿根本輕鬆平常 (然後就..)\n要知道性慾是生物本能 大傳康輔吉他熱音...\n這幾的社團的人 真的是脫魯比例最高的\n但這還是人的問題 那些魯的始終會魯\n這是一個大傳社員可以告訴你的\n\n如果真的因為癢到受不了進大傳\n現在就不會選擇後來的升學道路 何苦呢\n因為我最看重的不是這個\n但真的有抗癢性較差的 就真的啥活動都參加\n高中三年每一年看到都是不同的女伴 真的是牛逼\n園遊會一票妹子拉到攤位前喊人名 真的有面子\n\n

### Verification 2_2

In [56]:
# MUST Run this to verify your code
# You cannot modify the following code
# should be larger than 100

print(len(all_post))
df_ptt = pd.DataFrame(all_post)
print(len(set(df_ptt.link)))

200
200


### Verification 2_3

In [57]:
# MUST Run this to verify your code
# You cannot modify the following code

unnest = df_ptt.explode('comments')
unnest

,author,link,title,timestamp,content,comments
0,yoche2000 (九龍塘麵包粉),https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A...,Re: [討論] 自己經驗來說 贊同自己小孩唸男校女校嗎 - 看板 Boy-Girl - 批...,Sat May 8 15:29:02 2021,對選男校女校的人來說 重點是校風還有資源\n以一個男校仔的觀點 男女校對感情根本沒差\n\n...,"{'tag': '→ ', 'userid': 'Levesque', 'content':..."
0,yoche2000 (九龍塘麵包粉),https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A...,Re: [討論] 自己經驗來說 贊同自己小孩唸男校女校嗎 - 看板 Boy-Girl - 批...,Sat May 8 15:29:02 2021,對選男校女校的人來說 重點是校風還有資源\n以一個男校仔的觀點 男女校對感情根本沒差\n\n...,"{'tag': '→ ', 'userid': 'Levesque', 'content':..."
0,yoche2000 (九龍塘麵包粉),https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A...,Re: [討論] 自己經驗來說 贊同自己小孩唸男校女校嗎 - 看板 Boy-Girl - 批...,Sat May 8 15:29:02 2021,對選男校女校的人來說 重點是校風還有資源\n以一個男校仔的觀點 男女校對感情根本沒差\n\n...,"{'tag': '推 ', 'userid': 'rock801121', 'content..."
0,yoche2000 (九龍塘麵包粉),https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A...,Re: [討論] 自己經驗來說 贊同自己小孩唸男校女校嗎 - 看板 Boy-Girl - 批...,Sat May 8 15:29:02 2021,對選男校女校的人來說 重點是校風還有資源\n以一個男校仔的觀點 男女校對感情根本沒差\n\n...,"{'tag': '→ ', 'userid': 'rock801121', 'content..."
0,yoche2000 (九龍塘麵包粉),https://www.ptt.cc/bbs/Boy-Girl/M.1620458945.A...,Re: [討論] 自己經驗來說 贊同自己小孩唸男校女校嗎 - 看板 Boy-Girl - 批...,Sat May 8 15:29:02 2021,對選男校女校的人來說 重點是校風還有資源\n以一個男校仔的觀點 男女校對感情根本沒差\n\n...,"{'tag': '→ ', 'userid': 'yoche2000', 'content'..."
...,...,...,...,...,...,...
199,Kathy92 (士林王祖賢),https://www.ptt.cc/bbs/Boy-Girl/M.1619793014.A...,[求助] 分手後還維持同居 - 看板 Boy-Girl - 批踢踢實業坊,Fri Apr 30 22:30:12 2021,跟對方交往四年了\n\n三月底的時候因為家務小問題起了爭執\n\n爭執完後我就出門上班了\n...,"{'tag': '→ ', 'userid': 'kenichisun', 'content..."
199,Kathy92 (士林王祖賢),https://www.ptt.cc/bbs/Boy-Girl/M.1619793014.A...,[求助] 分手後還維持同居 - 看板 Boy-Girl - 批踢踢實業坊,Fri Apr 30 22:30:12 2021,跟對方交往四年了\n\n三月底的時候因為家務小問題起了爭執\n\n爭執完後我就出門上班了\n...,"{'tag': '→ ', 'userid': 'kenichisun', 'content..."
199,Kathy92 (士林王祖賢),https://www.ptt.cc/bbs/Boy-Girl/M.1619793014.A...,[求助] 分手後還維持同居 - 看板 Boy-Girl - 批踢踢實業坊,Fri Apr 30 22:30:12 2021,跟對方交往四年了\n\n三月底的時候因為家務小問題起了爭執\n\n爭執完後我就出門上班了\n...,"{'tag': '→ ', 'userid': 'kenichisun', 'content..."
199,Kathy92 (士林王祖賢),https://www.ptt.cc/bbs/Boy-Girl/M.1619793014.A...,[求助] 分手後還維持同居 - 看板 Boy-Girl - 批踢踢實業坊,Fri Apr 30 22:30:12 2021,跟對方交往四年了\n\n三月底的時候因為家務小問題起了爭執\n\n爭執完後我就出門上班了\n...,"{'tag': '推 ', 'userid': 'shawncarter', 'conten..."


In [58]:
# should be (1xxxxx, 6), e.g, (11393, 6)

unnest.shape

(16821, 6)

# AS05_3 News crawler
挑選apple daily news、udn online news或ltn online news任一家，這三家線上新聞的「搜尋頁面」其搜尋結果都是用JSON傳回前端，請挑選一則新聞撰寫爬蟲，撰寫完後轉為pandas，並列印出該pandas的shape。爬蟲條件要求：
1. 每則新聞應該有標題、鏈結、內文。亦即除了爬取新聞標題、鏈結外，還需要根據鏈結來爬取內文。
2. 新聞數應不少於100則。




In [158]:
#先回傳的資料做整理
url = 'https://udn.com/api/more?page=2&id=2&channelId=2&cateId=0&last_page=64&type=rank&sort=pv'
data2 = requests.get(url).json()
print(type(data2))
print(data2.keys())
# print(data2['state'])
# print(data2['page'])
# print(data2['end'])
print(type(data2['lists']))
print(len(data2['lists']))
print(data2['lists'][0])
print(data2['lists'][0].keys())

<class 'dict'>
dict_keys(['state', 'page', 'end', 'lists'])
<class 'list'>
20
{'url': 'https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2021/11/18/realtime/14578766.jpg&s=Y&x=0&y=108&sw=1278&sh=852&exp=3600', 'titleLink': 'https://udn.com/news/story/7320/5898751', 'title': '影／黃珊珊帶隊臨檢制服酒店 現場女幹部激動合影', 'paragraph': '酒店復業第3天，台北市副市長黃珊珊今天凌晨帶隊臨檢台北市警中山分局臨檢一家制服店，現場告訴業者，警方近日已啟動「安城專案...', 'cateLink': 'https://udn.com/news/cate/2/6639', 'cateTitle': '社會', 'time': {'dateTime': '2021-11-18 01:32', 'date': '2021-11-18 01:32'}, 'view': 13388}
dict_keys(['url', 'titleLink', 'title', 'paragraph', 'cateLink', 'cateTitle', 'time', 'view'])


In [159]:
#先把每篇的標題、url、作者給存起來
all_data = []
for i in range(10):
    url = 'https://udn.com/api/more?page=' + str(i) + '&id=2&channelId=2&cateId=0&last_page=64&type=rank&sort=pv'
    data = requests.get(url).json()
    for raw in data['lists']:
        title = raw['title']
        link = raw['titleLink']
        all_data.append({'title' : title,
                         'link' : link})
    if len(all_data) > 100:
        break

# print(all_data)

In [162]:
import pandas as pd
from bs4 import NavigableString
url_now = all_data[11]['link']
print(url_now)
res = requests.get(url_now, timeout=(1, 2))
soup = BeautifulSoup(res.text, "html.parser")
# print(soup)
content = soup.find(class_ = "article-content__editor").text.strip()
print(content)

# print(len(soup.find(class_ = "article-content__editor").text.strip()))
# print(soup.find(class_ = "article-content__editor").find_all("p"))
# print(type(soup.find(class_ = "article-content__editor").find_all("p")))
# print(content)
# print(soup.find(class_ = "article-content__editor"))
# print('-'*80)
# print(type(soup.find(class_ = "article-content__editor").text))

https://udn.com/news/story/122190/5899503
台北市萬芳醫院精神科醫師潘建志今在臉書分享，台灣第一例、兩劑高端混打AZ加莫德納完整四劑的真實個案，「新四劑高端戰士誕生」，潘指出，個案為第一線高風險醫護人員，自9月初混打至迄今，身體情況大致正常，從未出現血栓或心肌炎相關症狀，但也提出「最奇怪的地方」。

潘建志指出，個案為北市第一線高風險醫護人員，負責照顧新冠重症病患，2月底參加高端臨床實驗打第一劑、3月底打完第二劑，當時健保卡上並未出現疫苗註記。












台北市長柯文哲6月要求北市醫護強制接種疫苗，當時高端尚未解盲，個案雲端健保卡又沒有任何疫苗註記，不知道自己打到的是疫苗或安慰劑生理食鹽水，因此院方安排他接受兩種快篩抗體測試，結果均為陰性。

院方認為個案被隨機分配到安慰劑組，於是個案在6月初接種AZ疫苗、9月初接種莫德納，當時的政策有開放醫護AZ混打莫德納。

潘建志表示，高端之後解盲，個案很意外被主持實驗的醫師告知，確定他參加的是疫苗組，同時雲端健保卡上也出現高端註記，兩劑都是有批號的疫苗（安慰劑組生理食鹽水沒有批號不會註記）。

至此可確定該個案打了四劑真疫苗，分別為高端、高端、AZ、莫德納，潘建志說，公開這些資訊有獲得個案本人同意，也希望對70多萬名高端接種者有幫助。

潘表示，個案從9月初至迄今，身體情況大致正常，從未出現血栓或心肌炎相關症狀，後續抽血檢查，包含血小板、D-dimer指數、心肌發炎指數、肝腎功能指數都在正常範圍。

該個案例常例常鼻咽拭子採檢PCR數，10次都是陰性，目前持續上班工作中。近期自費接受抗S蛋白的抗體檢查，結果是陽性，顯示打AZ+莫德納後終於有抗體產生，但「最奇怪的地方在於，6月接受的抗體檢查，兩次均為陰性」。

潘建志指出，這位新四劑高端戰士「零號機」年紀尚輕、小於30歲，不是年紀大的病患免疫功能不好，且後來加打的AZ+莫德納也出現抗體，所以也不是先天免疫功能缺失的病患，「那是怎麼回事，兩劑高端的抗體跑哪裡去了？」

潘質疑，間隔2個月，血液中的抗體會下降這麼快？兩劑高端+AZ+莫德納，血液裡同時充滿S蛋白、mRNA和DNA，可說是最強人類補完計畫。

針對其他打高端有出國需求的人，是否該再接種兩劑國際疫苗及有無風險？潘建志認為，一般民眾自己難以判斷，應交給專業，由醫師評估

In [163]:
n = 1
for x in all_data:
    url = x['link']
    res = requests.get(url, timeout=(1, 2))
    soup = BeautifulSoup(res.text, "html.parser")
    # print(soup)
    content = soup.find(class_ = "article-content__editor").text.strip()
    x['main_content'] = content
    # print(str(n) + 'done')
    # n += 1
    
df_all_data = pd.DataFrame(all_data)
df_all_data

,title,link,main_content
0,華南清涼照桌曆驚動金管會 張振山：商業行為需符期待,https://udn.com/news/story/7239/5899088,華南金控（2880）旗下華南期貨送客戶2022年新桌曆竟用員工「清涼照」，由於封面七位美女照...
1,晚一小時就有差 研究顯示：「黃金時段」入睡可救你一命,https://udn.com/news/story/7266/5897068,英國生物銀行（UK Biobank）調查88026人的就寢時間後發現，晚間10到11點是最好...
2,影／台1線枋山段重機「插進賓士」 騎士無生命跡象,https://udn.com/news/story/7320/5900524,省道台1線屏東縣枋山鄉中山路3段北上車道今天下午發生重機車撞上賓士轎車車故，救護人員趕到現場...
3,LINE隱藏功能「傳訊快好幾倍」？ 全網驚：我還真的不知道,https://udn.com/news/story/11017/5898041,LINE是不少台灣人每天最常使用的通訊軟體，但不是每個人對手機系統或APP的操作都暸若指掌，...
4,高端族可再混打國際疫苗 孫大千：台灣獨步全球6沒有,https://udn.com/news/story/122190/5899552,中央流行疫情指揮中心開放接種國產高端疫苗民眾，若有緊急出國需求，可再混打國際認證疫苗，引發爭...
...,...,...,...
115,女主播老公狼牙棒打姪子滿臉血 判決定讞結果出爐！,https://udn.com/news/story/7321/5899349,主播王佳婉丈夫、捷年集團董事長呂台年不滿侄子劉鎮亞態度差，兩人口角，他事後持狼牙棒手電筒毆打...
116,太電前董座孫道存縱橫商場 起伏人生,https://udn.com/news/story/122564/5898443,太平洋電線電纜公司前董事長孫道存昨（17）日因癌症過世，他縱橫商場數十年，經營的太電曾是台灣...
117,NBA／湖人宣佈裁掉唐布亞 小將布朗獲雙向合約回歸,https://udn.com/news/story/7002/5899614,籃網先前為了節省天價高額豪華稅，決定將從活塞交易所得的唐布亞（Sekou Doumbouya...
118,影／休旅車國道突右切！轎車失控撞護欄狂轉 車頭全爛,https://udn.com/news/story/7320/5899319,國道一號彰化田尾路段約215.8公里附近，17日下午5時許發生一起事故，一輛高速行駛的黑色自...
